<a href="https://colab.research.google.com/github/eastrong23/Works/blob/main/DM_database_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from google.colab import files

## Loading the Data

In [ ]:
# --------------------------------------
# STEP 1: Connect to a new SQLite database
# --------------------------------------
conn = sqlite3.connect("static_retail.db")
cursor = conn.cursor()
print("Connected to static_retail.db")

# --------------------------------------
# STEP 2: Load CSV files into SQLite tables
# (Assuming the CSV files are already uploaded to Colab)
# --------------------------------------
customers_df   = pd.read_csv("Customers.csv")
products_df    = pd.read_csv("Products.csv")
salesreps_df   = pd.read_csv("SalesReps.csv")
campaigns_df   = pd.read_csv("Campaigns.csv")
orders_df      = pd.read_csv("Orders.csv")
order_items_df = pd.read_csv("Order_Items.csv")



Connected to static_retail.db


In [ ]:
conn = sqlite3.connect("static_retail.db")
cursor = conn.cursor()

print("Connected to static_retail.db")
cursor.execute("""
Create table Customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(50),
    email VARCHAR(100),
    address VARCHAR(100),
    registration_date TIMESTAMP
);
""")
cursor.execute("""
Create table Products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(50),
    category VARCHAR(50),
    price DECIMAL(5,1)
);
""")
cursor.execute("""
Create table SalesReps (
    sales_rep_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(50),
    region VARCHAR(50)
);
""")
cursor.execute("""
Create table Campaigns (
    campaign_id INTEGER PRIMARY KEY AUTOINCREMENT,
    campaign_name VARCHAR(50),
    start_date TIMESTAMP,
    end_date TIMESTAMP,
    budget INT
);
""")
cursor.execute("""
Create table Orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INT,
    sales_rep_id INT,
    campaign_id INT,
    order_date TIMESTAMP,
    total_amount DECIMAL(5,1),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (sales_rep_id) REFERENCES SalesReps(sales_rep_id),
    FOREIGN KEY (campaign_id) REFERENCES Campaigns(campaign_id)
);
""")
cursor.execute("""
CREATE table Order_Items(
  order_id INT,
  product_id INT,
  quantity INT,
  item_total DECIMAL(5,1),
  FOREIGN KEY (order_id) REFERENCES Orders(order_id),
  FOREIGN KEY (product_id) REFERENCES Products(product_id)
);""")

conn.commit()
print("Table Constructed")
# Write DataFrames to SQLite tables (replace any existing table)
customers_df.to_sql("Customers", conn, if_exists="append", index=False)
products_df.to_sql("Products", conn, if_exists="append", index=False)
salesreps_df.to_sql("SalesReps", conn, if_exists="append", index=False)
campaigns_df.to_sql("Campaigns", conn, if_exists="append", index=False)
orders_df.to_sql("Orders", conn, if_exists="append", index=False)
order_items_df.to_sql("Order_Items", conn, if_exists="append", index=False)

print("CSV files loaded into SQLite tables.\n")

Connected to static_retail.db
Table Constructed
CSV files loaded into SQLite tables.



In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)

Table: Customers
  Column: customer_id, Type: INTEGER, NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: name, Type: VARCHAR(50), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: email, Type: VARCHAR(100), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: address, Type: VARCHAR(100), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: registration_date, Type: TIMESTAMP, NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: sqlite_sequence
  Column: name, Type: , NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: seq, Type: , NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: Products
  Column: product_id, Type: INTEGER, NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: product_name, Type: VARCHAR(50), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: category, Type: VARCHAR(50), NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: price, Type: DECIMAL(5,1), NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------

In [ ]:
pd.read_sql_query("SELECT * FROM Order_Items LIMIT 5;", conn)

,order_id,product_id,quantity,item_total
0,1,16,4,184.2
1,2,36,4,47.6
2,3,26,3,736.0
3,3,15,1,868.8
4,3,44,1,132.2


## Sales Trend Report
This query aggregates orders by month to show overall trends in order count and revenue.

Insights:

*   Helps identify peak sales periods and seasonal trends.
*   Informs forecasting and inventory planning.

In [ ]:
# --------------------------------------
# STEP 3: Execute SQL queries for business reports
# --------------------------------------

# Query 1: Sales Trend Report (aggregating orders by month)
sales_trend_query = """
SELECT
    strftime('%Y-%m', order_date) AS Month,
    COUNT(order_id) AS Total_Orders,
    SUM(total_amount) AS Total_Revenue
FROM Orders
GROUP BY strftime('%Y-%m', order_date)
ORDER BY Month;
"""
df_sales_trend = pd.read_sql_query(sales_trend_query, conn)
print("Sales Trend Report:")
print(df_sales_trend)
print("\n")

Sales Trend Report:
      Month  Total_Orders  Total_Revenue
0   2024-03            20         1169.7
1   2024-04            47         3030.5
2   2024-05            31         1878.3
3   2024-06            36         2866.0
4   2024-07            52         3238.4
5   2024-08            40         2977.2
6   2024-09            57         4111.5
7   2024-10            49         3566.8
8   2024-11            35         2427.2
9   2024-12            38         2726.4
10  2025-01            43         3310.1
11  2025-02            35         2473.4
12  2025-03            17         1064.7




## Campaign Impact Report
This query measures the effectiveness of marketing campaigns by summarizing the number of orders, total revenue, and average order value per campaign.

Insights:

* Shows which campaigns generate the most orders and revenue.
* Supports decision-making on future marketing investments.



In [ ]:
# Query 2: Campaign Impact Report (joining Orders with Campaigns)
campaign_impact_query = """
SELECT
    cmp.campaign_id,
    cmp.campaign_name,
    COUNT(o.order_id) AS Orders_Count,
    SUM(o.total_amount) AS Revenue,
    AVG(o.total_amount) AS Avg_Order_Value
FROM Orders o
JOIN Campaigns cmp ON o.campaign_id = cmp.campaign_id
GROUP BY cmp.campaign_id, cmp.campaign_name
ORDER BY Revenue DESC;
"""
df_campaign_impact = pd.read_sql_query(campaign_impact_query, conn)
print("Campaign Impact Report:")
print(df_campaign_impact)
print("\n")

Campaign Impact Report:
   campaign_id campaign_name  Orders_Count  Revenue  Avg_Order_Value
0            3    Campaign 3            59   4089.3        69.310169
1           10   Campaign 10            54   3862.4        71.525926
2            8    Campaign 8            54   3786.4        70.118519
3            6    Campaign 6            55   3738.4        67.970909
4            5    Campaign 5            57   3733.2        65.494737
5            1    Campaign 1            47   3380.3        71.921277
6            9    Campaign 9            47   3372.4        71.753191
7            7    Campaign 7            42   3080.5        73.345238
8            2    Campaign 2            44   3029.3        68.847727
9            4    Campaign 4            41   2768.0        67.512195




## Sales Representative Performance Report
This query evaluates sales representative performance by summarizing their order count, total revenue, and average order value.

Insights:

* Identifies top-performing sales representatives.
* Provides data to support performance-based incentives or targeted training.



In [ ]:
# Query 3: Sales Representative Performance Report (joining Orders with SalesReps)
sales_rep_query = """
SELECT
    sr.sales_rep_id,
    sr.name AS Sales_Rep_Name,
    COUNT(o.order_id) AS Orders_Count,
    SUM(o.total_amount) AS Total_Revenue,
    AVG(o.total_amount) AS Avg_Order_Value
FROM Orders o
JOIN SalesReps sr ON o.sales_rep_id = sr.sales_rep_id
GROUP BY sr.sales_rep_id, sr.name
ORDER BY Total_Revenue DESC;
"""
df_sales_rep = pd.read_sql_query(sales_rep_query, conn)
print("Sales Representative Performance Report:")
print(df_sales_rep)
print("\n")

Sales Representative Performance Report:
    sales_rep_id Sales_Rep_Name  Orders_Count  Total_Revenue  Avg_Order_Value
0             14    SalesRep 14            32         2255.6        70.487500
1             11    SalesRep 11            29         2183.4        75.289655
2             10    SalesRep 10            30         2087.9        69.596667
3             19    SalesRep 19            27         1970.4        72.977778
4             20    SalesRep 20            25         1934.2        77.368000
5              2     SalesRep 2            29         1900.2        65.524138
6              3     SalesRep 3            27         1875.0        69.444444
7             17    SalesRep 17            29         1869.6        64.468966
8              8     SalesRep 8            27         1866.9        69.144444
9             16    SalesRep 16            29         1852.3        63.872414
10            18    SalesRep 18            25         1844.7        73.788000
11            13    Sal

## Customer Spending Report
This query segments customers based on their spending, showing total orders, total spending, and average order value.

Insights:

* Highlights high-value customers for targeted loyalty programs.
* Helps in identifying potential customer segments.



In [ ]:
# Query 4: Customer Spending Report (joining Orders with Customers)
customer_spending_query = """
SELECT
    c.customer_id,
    c.name AS Customer_Name,
    COUNT(o.order_id) AS Orders_Count,
    SUM(o.total_amount) AS Total_Spending,
    AVG(o.total_amount) AS Avg_Order_Value
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.name
ORDER BY Total_Spending DESC;
"""
df_customer_spending = pd.read_sql_query(customer_spending_query, conn)
print("Customer Spending Report:")
print(df_customer_spending)
print("\n")


Customer Spending Report:
     customer_id Customer_Name  Orders_Count  Total_Spending  Avg_Order_Value
0            442  Customer 442             4           351.7           87.925
1            289  Customer 289             4           343.2           85.800
2            176  Customer 176             4           322.0           80.500
3            152  Customer 152             5           299.4           59.880
4             47   Customer 47             4           286.1           71.525
..           ...           ...           ...             ...              ...
304            5    Customer 5             1            21.4           21.400
305          394  Customer 394             1            21.2           21.200
306          168  Customer 168             1            21.0           21.000
307          210  Customer 210             1            21.0           21.000
308          439  Customer 439             1            20.7           20.700

[309 rows x 5 columns]




## Product Sales Performance Report
This query aggregates order item data to determine which products are the best sellers.

Insights:

*  Reveals the top-selling products.
*  Informs product promotion and inventory management.




In [ ]:
# Query 5: Product Sales Performance Report (joining Order_Items with Products)
product_sales_query = """
SELECT
    p.product_id,
    p.product_name,
    SUM(oi.quantity) AS Total_Quantity_Sold,
    SUM(oi.item_total) AS Total_Sales
FROM Order_Items oi
JOIN Products p ON oi.product_id = p.product_id
GROUP BY p.product_id, p.product_name
ORDER BY Total_Quantity_Sold DESC;
"""
df_product_sales = pd.read_sql_query(product_sales_query, conn)
print("Product Sales Performance Report:")
print(df_product_sales)
print("\n")

Product Sales Performance Report:
    product_id product_name  Total_Quantity_Sold  Total_Sales
0           38   Product 38                  172       9697.6
1           10   Product 10                  165       8336.0
2           36   Product 36                  163       8357.6
3           12   Product 12                  157       9587.8
4           40   Product 40                  152       9187.3
5           50   Product 50                  151       7649.3
6           22   Product 22                  141      10180.1
7           16   Product 16                  137      10167.5
8           17   Product 17                  134       8991.2
9           32   Product 32                  134       7510.7
10          28   Product 28                  133      11234.6
11           6    Product 6                  130       6100.0
12          44   Product 44                  130       5810.4
13           1    Product 1                  128       6498.1
14          19   Product 19         